# A Public Health Dataset

## Background

This report was created by Geethanjali Krishnappa, Kevin Parrott, Jacob Rosen, and Manoj Venkatachalaiah for DSCS 511, Spring 2019. 

In this project, our group extracted data from two public health data sources and turned it into a single dataset. The inspiration for this was seeing two maps, [here](https://www.cdc.gov/heartdisease/maps_data.htm) (CDC) and [here](https://findahealthcenter.hrsa.gov/) (HRSA), that were very good visual tools but unusable for analysis. For example, if a researcher wanted to model the affect of obesity and diabetes on the amount of people hospitalized for heart disease, they would need to manually collect the data from this map or the individual sources the map used. Instead, our group used webscraping to get the data programmatically.

These are all the packages that are required. If you don't have them then go get them.

In [2]:
#Loading all the neccessary modules
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import urllib
# from pprint import pprint
import pandas as pd
# import json
import re
import csv
from collections import Counter

## FIPS Dictionary

Every data point we want to collect is a piece of information representing a county. Each county can be identified by its name, or more specifically its FIPS (Federal Information Processing Standard Publication) code. Therefore, we are going to create a nested dictionary for each state so we can easily access identifying information. This will be very useful when we build dataframes in later sections. 

[Website Reference](http://datadictionary.naaccr.org/?c=11)

We ended up utilizing a few different sources for FIPS codes throughout the process, but the main source was a website that listed all of the FIPS Codes for US states and territories. There were some inconsistencies between sources that led to problems when merging data sets.

##### Step 1) Scrape

The first of many `BeautifulSoup` scrapes. Luckily this site is static.

In [3]:
#creates skin for nested dictionary
scraped_fips_dict = {}

#requests website URL
html_text = urllib.request.urlopen("http://datadictionary.naaccr.org/?c=11").read()

#creates a beautiful soup object
soup = BeautifulSoup(html_text, 'html.parser')

##### Step 2) Filter and Format County Names

In [4]:
#loop that collects all of the useful data

tablebody = soup.find_all('div',{'class':'tableAppendixBody'})

tablebodylist=[]
for body in tablebody:
    county_code = body.find('div',{'class':'code'}).text
    county_code = re.findall('[0-9]{3}',county_code)
    county_name = body.find('div',{'class':'title'}).text
    if '(city)' in county_name:
        county_name = re.findall('[A-Za-z- .]+[a-z]',county_name)
        county_name = county_name[0]
        county_name = county_name+' City'
    if '(City)' in county_name:
        county_name = re.findall('[A-Za-z- .]+[a-z]',county_name)
        county_name = county_name[0]
        county_name = county_name+' City'
    else:
        county_name = re.findall('[A-Za-z- .]+[a-z]',county_name)
        county_name = county_name[0]
        county_name = county_name.replace(' County','')
    tablebodylist.append([county_name,county_code[0]])


##### Step 3) Create a list of all the State/Territory info

In [5]:
tableheadlist = []
tablehead = soup.find_all('strong')
for head in tablehead:
    head = head.text
    if len(head) >20:
        state = re.findall('NAME: ([A-Za-z .]+[a-z]) ',head)
        abbv = re.findall('ALPHABETIC CODE: ([A-Z]{2})',head)
        state_code = re.findall('NUMERIC CODE: ([0-9]{2})',head)
        tableheadlist.append([state[0],abbv[0],state_code[0]])

##### Step 4) Create a list that tells a loop when to cut off for each state

In [6]:
# a list of endpoints that dreaks up the states/territories
endlist=[['Kusilvak', '158'],['Winston', '133'],['Yell', '149'],['Western', '050'],\
        ['Yuma', '027'],['Yuba', '115'],['Yuma', '125'],['Windham', '015'],['District of Columbia', '001'],\
        ['Sussex', '005'],['Washington', '133'],['Yap', '060'],['Worth', '321'],['Guam', '010'],\
        ['Maui', '009'],['Wright', '197'],['Washington', '087'],['Woodford', '203'],\
        ['Whitley', '183'],['Wyandotte', '209'],['Woodford', '239'],['Winn', '127'],\
        ['Worcester', '027'], ['Baltimore', '510'],['York', '031'],['Wotle', '430'],\
        ['Wexford', '165'],['Yellow Medicine', '173'],['St. Louis City', '510'],['Tinian', '120'],\
        ['Yazoo', '163'],['Yellowstone', '111'],['Yancey', '199'],['Williams', '105'],\
        ['York', '185'],['Sullivan', '019'],['Warren', '041'],['Valencia', '061'],['Carson City', '510'],\
        ['Yates', '123'],['Wyandot', '175'],['Woodward', '153'],['Yamhill', '071'],['York', '133'],\
        ['Yauco', '153'],['Sonsorol', '370'],['Washington', '009'],['York', '091'],['Oglala Lakota', '102'],\
        ['Wilson', '189'],['Zavala', '507'],['Wake Island', '450'],['Weber', '057'],['Winchester City', '840'],\
        ['St. Thomas', '030'],['Windsor', '027'],['Yakima', '077'],['Wood', '141'],\
        ['Wyoming', '109'],['Weston', '045']]

##### Step 5) Create nested dictionaries for each state

In [7]:
#Creates the skeleton for the dictionary
for state in tableheadlist:
    scraped_fips_dict[state[1]]={'State/Province':state[0],'Counties':{}}

##### Step 6) Add in County Names and FIPS Codes

In [8]:
#fills the values into the dictionary
stateindex=0

for i,county in enumerate(tablebodylist):
    state_code=tableheadlist[stateindex][2]
    if county[0]!=endlist[stateindex][0]:
        county_code=county[1]
        scraped_fips_dict[tableheadlist[stateindex][1]]['Counties'][county[0]]={'FIPS':state_code+county_code}
    if county[0]==endlist[stateindex][0]:
        county_code=county[1]
        scraped_fips_dict[tableheadlist[stateindex][1]]['Counties'][county[0]]={'FIPS':state_code+county_code}
        stateindex+=1

## Webscraping

Initially webscraping was actually the plan B to our original idea of requesting access to a restricted CDC file. We were going to apply, hopefully get access, and then use some fancy CSV parser to clean the data. But for various reasons - similar parsing had already been done, review process taking too long, etc. - we chose to scrape data from the interactive map. Truthfully, this is misleading as the webpage we actually scraped was [here](https://nccd.cdc.gov/DHDSPAtlas/Reports.aspx). The tricky thing about this webpage is that it uses ASPX. Unlike normal HTML which is static, ASPX allows the website to be interactive. This is great for normal users but difficult if the goal is scraping. While in HTML the url changes every time new the page displays new information, the url for an ASPX page does not. Therefore, the only way to access the information programmatically is to run code that simulates the necessary input for the desired information to be loaded. From there, it is simply getting the html, parsing, and other basic BS4 actions. 

##### Step 1)  Selenium 
Install Selenium. 

In [8]:
# !pip install selenium
# !pip install webdriver-manager

Install the driver for your favorite web browser (in our case Chrome).

In [9]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager

# webdriver.Chrome(ChromeDriverManager().install())

After downloading the driver you'll need to go to the location given in the output and extract the driver executable to some location. The path to this location will be important in the next section.

Selenium itself is actually a suite of different software; the specific one used here is called WebDriver. This tool is generally used to test websites, but its ability to programmatically interact with the elements of the site is why it is necessary on top of BeautifulSoup. To understand what it is doing, we will borrow a great explanation created by a Quora user [here](https://www.quora.com/How-does-the-Selenium-WebDriver-work) (feel free to read the hard version on your own). Imagine a taxi cab ride; there are 3 components- rider, driver, car. The rider gives instructions, the driver inputs them into the car, and the car executes. In the case of Selenium, the rider is the user, the driver is the browser driver, and the car is the browser. The user types instructions in code, the driver sends those requests to the browser, and the browser exexutes those requests. 

##### Step 2)  Selenium Browser
These commands show webdriver where the browser and driver are located in the computer so that it can pull up a webpage in that browser. A new window will pop up and say "*Browser is being controlled by automated test software*".

Each code block in the following cell does the same thing; the location paths are for each of our group member's computers.

In [9]:
#Jacob 
options = webdriver.ChromeOptions()
options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
chrome_driver_binary = r"C:\Users\Jacob Rosen\Desktop\chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)

#Kevin
# options = webdriver.ChromeOptions()
# options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
# chrome_driver_binary = r"C:\Users\parro\Desktop\chromedriver.exe"
# driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)

#Manoj
# options = webdriver.ChromeOptions()
# options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
# chrome_driver_binary = r"C:\Users\manoj v\desktop\chromedriver.exe"
# driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)

C:\Users\Jacob Rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


##### Step 3)  CDC Website
Pass the url of the page you want to access to the driver. 

In [10]:
url = 'https://nccd.cdc.gov/DHDSPAtlas/Reports.aspx'
driver.get(url)

##### Step 4) Page Defaults
This page actually has defaults already set so we can simply click *Show Results* to get data. Clicking county statistics is not necessary because the data is already loaded in the html.

In [11]:
submit_button = driver.find_element_by_xpath('//*[@id="btn_report_submit"]')
submit_button.click()

**!** Sometimes you might get an error saying "element is not interactable". Try running the above code a few times; if it does't work there might be a deeper problem.

From here we can immediately create a data table. Unlike the later ones, this one contains extra columns like names because it will be the table all others will be merged to. 

In [12]:
#Loops through and grabs the important data
data_list=[]
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')
report_body= soup.find('tbody',{'id':'report_body'})
row_list=report_body.find_all('tr')

for row in row_list:
    county_name=row.find('a')
    county_name=county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    state= scraped_fips_dict[abbv]['State/Province']
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,county_name,abbv,state,data]
    data_list.append(line)
DiseaseRateTable = pd.DataFrame(data_list,columns = ["FIPS Code","County Name","State Abbreviation","State Name","CV Death per 100K"])
DiseaseRateTable

,FIPS Code,County Name,State Abbreviation,State Name,CV Death per 100K
0,02013,Aleutians East,AK,Alaska,106.1
1,08097,Pitkin,CO,Colorado,133.8
2,08037,Eagle,CO,Colorado,157.5
3,08117,Summit,CO,Colorado,171.8
4,02016,Aleutians West,AK,Alaska,184.7
5,08113,San Miguel,CO,Colorado,186.0
6,08091,Ouray,CO,Colorado,210.9
7,56039,Teton,WY,Wyoming,215.2
8,51685,Manassas Park,VA,Virginia,221.1
9,16013,Blaine,ID,Idaho,234.1


##### Step 5)  Selectors
But if we only wanted one type of data we could've downloaded it manually from the map. The code below can access every type of interactive element on the page related to loading data- tabs, bubble menus, drop-down menus. The only difference between categories will be specifics like name and xpath.

The driver variable has many ways to search for elements within the html. We chose to search by Xpath. Xpath stands for XML path and is used to find elements in XML using an XML path expression. This is helpful when things like ID or tag are unhelpful for finding specific elements. It can be accessed by right clicking on the element in the html and going to `Copy -> Copy Xpath`. 

The first code block clicks the *Social and Economic tag*. The second block selects an option from the bubble menus. The third code selects an option from a drop down menu. There are actually more drop-downs under the *Health Indicators* tab, but they all follow the same selector-option structure. This page is now prepared to load data on the percentage of the population aged 65 and older. 


In [13]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Bubble menu- Demographics
bubble = driver.find_element_by_xpath('//*[@id="2_option_13"]')
bubble.click()

#select drop down menu- Aged 65 and Older (%)
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[6]')
dropdown.click()


##### Step 6)  Results
Once we have specified what data we want, we can click the *Show Results* button again to load the data. Notice that the Xpath to the *Show Results* button for the Social and Economic data is different than the button for the default tab *Health Indicators*.

In [14]:
#show results
results = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
results.click()

##### Step 7)  BeautifulSoup

Now that the data is actually in the page's html, we can scrape it like any other static page with BeautifulSoup. 



In [24]:
html=driver.page_source
soup = BeautifulSoup(html, 'html.parser')

Finding what we want simply requires locating the correct tags. Then we can search by tag with the find methods built into bs4. 


In [25]:
report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')

##### Step 8) Build

The for loop below uses the parsed html and the FIPS dictionary we created earlier to create a list of labeled data that can be turned into a dataframe. It takes the data we want from each row of the html we scraped and uses the FIPS dictionary to assign that data to a FIPS code.



**Older Than 65 %**

In [26]:
old_data_list=[]

for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    old_data_list.append(line)

OldTable = pd.DataFrame(old_data_list,columns = ["FIPS Code","% Older than 65"])
OldTable

,FIPS Code,% Older than 65
0,13053,3.7
1,02016,5.5
2,02185,6.1
3,16065,6.3
4,46102,6.3
5,46121,6.9
6,02050,7.1
7,02188,7.2
8,38085,7.2
9,49049,7.2


We wanted to have a decently sized dataset as the final product for this report so we picked several more variables to collect. But as you will see both the data scraping and manipulation is very similar. Aside from a few things like variable names and fixing data originally stored in 1000s (like house price), all of this code is copy and paste.  

**!** The Selenium browser takes a few seconds to update. If you execute both cells of the table-making code too quickly then you will get a table identical to the one previously made.

**Population**

In [27]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Demographic Bubble menu
bubble = driver.find_element_by_xpath('//*[@id="2_option_13"]')
bubble.click()

#Population drop down menu
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[7]')
dropdown.click()

#show results
submit_button = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
submit_button.click()



In [28]:
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')
#if error given that report_body has no attribute findall, rerun the above cell and try again
Population_data_list=[]

report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')
for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    Population_data_list.append(line)

PopulationTable = pd.DataFrame(Population_data_list,columns = ["FIPS Code","Population"])
PopulationTable

,FIPS Code,Population
0,48301,74
1,15005,86
2,48269,289
3,31005,421
4,31117,432
5,30069,453
6,31009,502
7,31115,537
8,35021,546
9,48261,564


**Median Household Income**

In [29]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Social Evironment Bubble menu
bubble = driver.find_element_by_xpath('//*[@id="2_option_12"]')
bubble.click()

#Household Income drop down menu
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[6]')
dropdown.click()

#show results
submit_button = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
submit_button.click()


In [30]:
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')

#if error given that report_body has no attribute findall, rerun the above cell and try again
income_data_list=[]

report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')
for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    data=float(data)*1000
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    income_data_list.append(line)

IncomeRateTable = pd.DataFrame(income_data_list,columns = ["FIPS Code","Median Household Income"])
IncomeRateTable

,FIPS Code,Median Household Income
0,28051,22000.0
1,46017,22500.0
2,21189,23100.0
3,01131,24200.0
4,54047,24500.0
5,13259,24900.0
6,21051,24900.0
7,21147,25100.0
8,21095,25200.0
9,01105,25200.0


**Median Home Value**

In [31]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Social evironment Bubble menu
bubble = driver.find_element_by_xpath('//*[@id="2_option_12"]')
bubble.click()

#Home Value drop down menu
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[5]')
dropdown.click()

#show results
submit_button = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
submit_button.click()

In [32]:
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')
#if error given that report_body has no attribute findall, rerun the above cell and try again
home_data_list=[]

report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')
for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    data=float(data)*1000
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    home_data_list.append(line)



HomeValueTable = pd.DataFrame(home_data_list,columns = ["FIPS Code","Median Home Value"])
HomeValueTable

,FIPS Code,Median Home Value
0,46102,18700.0
1,48261,21300.0
2,48079,33300.0
3,54047,34800.0
4,48197,38400.0
5,46121,38500.0
6,48269,38800.0
7,48507,39900.0
8,48229,44100.0
9,48275,44400.0


**Food Stamp Recipient Percentage**

In [35]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Social evironment Bubble menu
bubble = driver.find_element_by_xpath('//*[@id="2_option_12"]')
bubble.click()

#Food Stamp drop down menu
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[4]')
dropdown.click()

#show results
submit_button = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
submit_button.click()



In [36]:
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')

#if error given that report_body has no attribute findall, rerun the above cell and try again
food_data_list=[]

report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')
for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    food_data_list.append(line)


FoodStampTable = pd.DataFrame(food_data_list,columns = ["FIPS Code","Food Stamp Recipient %"])
FoodStampTable

,FIPS Code,Food Stamp Recipient %
0,31005,0.4
1,56039,0.6
2,51610,0.9
3,48301,0.9
4,46119,1.1
5,30075,1.1
6,08097,1.2
7,08035,1.6
8,30055,1.6
9,56035,1.6


**Less than College Educated Percentage**

In [50]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Social evironment Bubble menu
bubble = driver.find_element_by_xpath('//*[@id="2_option_12"]')
bubble.click()

#College % drop down menu
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[2]')
dropdown.click()

#show results
submit_button = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
submit_button.click()

In [51]:
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')
#if error given that report_body has no attribute findall, rerun the above cell and try again
college_data_list=[]

report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')
for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    college_data_list.append(line)


CollegeTable = pd.DataFrame(college_data_list,columns = ["FIPS Code", "Edu. < College %"])
CollegeTable

,FIPS Code,Edu. < College %
0,51610,21.9
1,51013,25.9
2,35028,34.5
3,51510,38.2
4,51510,38.8
5,08097,38.8
6,36061,39.3
7,51600,39.3
8,08013,39.6
9,51107,40.2


**Less than High School Educated Percentage**

In [38]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Social evironment Bubble menu
bubble = driver.find_element_by_xpath('//*[@id="2_option_12"]')
bubble.click()

#Highschool % drop down menu
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[1]')
dropdown.click()

#show results
submit_button = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
submit_button.click()


In [39]:
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')
#if error given that report_body has no attribute findall, rerun the above cell and try again
highschool_data_list=[]

report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')
for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    highschool_data_list.append(line)

HighschoolTable = pd.DataFrame(highschool_data_list,columns = ["FIPS Code","Edu. < Highschool %"])
HighschoolTable

,FIPS Code,Edu. < Highschool %
0,31009,1.1
1,02068,1.3
2,31115,1.7
3,51610,1.8
4,49033,1.9
5,30069,1.9
6,35028,2.0
7,08047,2.0
8,08091,2.0
9,08019,2.1


**Poverty %**

In [42]:
#Social and Economic tab
tab = driver.find_element_by_xpath('//*[@id="report_type_tabs"]/ul/li[3]/a')
tab.click()

#Social evironment Bubble menu
bubble = driver.find_element_by_xpath('//*[@id="2_option_12"]')
bubble.click()

#Poverty drop down menu
dropdown = driver.find_element_by_xpath('//*[@id="2_theme_select"]/option[8]')
dropdown.click()

#show results
submit_button = driver.find_element_by_xpath('//*[@id="btn_2_report_submit"]')
submit_button.click()


In [43]:
html=driver.page_source

soup = BeautifulSoup(html, 'html.parser')
#if error given that report_body has no attribute findall, rerun the above cell and try again
poverty_data_list=[]

report_body= soup.find('tbody',{'id':'report_sd_body'})
row_list=report_body.find_all('tr')
for i,row in enumerate(row_list):
    county_name = row.find('th')
    county_name = county_name.text
    values=row.find_all('td')
    abbv=values[0].text
    data=values[1].text
    for name in scraped_fips_dict[abbv]['Counties']:
        if county_name in name:
            fips_code = scraped_fips_dict[abbv]['Counties'][name]['FIPS']
    line=[fips_code,data]
    poverty_data_list.append(line)


PovertyRateTable = pd.DataFrame(poverty_data_list,columns = ["FIPS Code","Poverty %"])
PovertyRateTable

,FIPS Code,Poverty %
0,08035,3.4
1,51107,3.5
2,51610,3.6
3,46083,3.7
4,35028,4.0
5,33015,4.2
6,27019,4.3
7,49029,4.3
8,27163,4.5
9,34019,4.5


## (Not) API Requests 

To supplement this data set, we wanted to get information on Community Health Centers. These are primary care delivery centers that get at least part of their funding from the federal government. Their purpose is to provie basic care to underserved populations. In a vey basic study, information about number of CHCs per county might be a rough but sufficient representation for how much resources are being provided to an area. 

We originally found this data available on an interactive [map](https://findahealthcenter.hrsa.gov/). And just like the CDC data there was no way to get all the data once. Luckily (or so we thought), there was an API with [documentation](https://datawarehouse.hrsa.gov/HDWDataServiceExternal/HdwDataWebServiceExternal.asmx?op=FindHealthCentersByArea) that should have given us access to that data and potentially other files from the Health Resources & Services Administration. They use a SOAP API which is different than what we used during class, and they even provided exmaple requests. But for reasons we never figured out we could not successfully query. The closest we got was getting a large XML file with elements of the interactive map page, but never the data we wanted. If we were really desparate, we probably could have used Selenium to programmatically interact with the map. However, we found the dataset available for download and took the [easy way out](https://data.hrsa.gov/data/download). The dataset contains a lot of information on individual health centers, but for our purposes we only need each CHC's county fips code to count health centers per county. 

#### What didn't work

Below is the code and output for the failed attempt at using SOAP requests on the Health Resources & Services Administration. 

In [ ]:
#this gives SVG output of all states
token = "73d40507-6765-4c6c-9e1f-1e8f06f9d3a4"
# <WhereClause>""</WhereClause>

import requests
# url="https://datawarehouse.hrsa.gov/HDWDataServiceExternal/HdwDataWebServiceExternal.asmx?WSDL"
url = "https://datawarehouse.hrsa.gov/webservices/FindHealthCentersByArea/HDWDataServiceExternal/HdwDataWebServiceExternal.asmx?WSDL"
#headers = {'content-type': 'application/soap+xml'}
headers = {'content-type': 'text/xml'}
body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <FindHealthCentersByArea xmlns="https://datawarehouse.hrsa.gov/webservices/">
      <StateFIPSCode>"48"</StateFIPSCode>
      <CountyFIPSCode>"48453"</CountyFIPSCode>
      <ZipCode>"78705"</ZipCode>
      <token>"73d40507-6765-4c6c-9e1f-1e8f06f9d3a4"</token>
    </FindHealthCentersByArea>
  </soap:Body>
</soap:Envelope>"""

response = requests.post(url,data=body,headers=headers)
# print(response.status_code)
# print(response.headers)
# print(response.content)
response.content

##### Step 1) CHC Location Data

We first checked if there were CHCs without fips codes. There turned out to be several, but the only ones with searchable addresses were two in California. There are many tools that can find county by address and so with those we were able to assign fips codes. The corrected data set will be attached with this report.

Load the location data. 

In [44]:
with open('CHC_location.csv') as file:
    reader = csv.reader(file)
    data=list(reader) 

##### Step 2) FIPS List

Find the column index that has the FIPS code for every CHC and Use it to create a list. Each instance of FIPS code in the data set corresponds to one CHC.

In [45]:
fips=[]
for i in data[1:]:
    fips.append(i[43])

##### Step 3) Count

Use the `Counter` method to go count each instance of unique FIPS codes. This gives the number of CHCs per county.

The `Counter` method outputs a dictionary; the FIPS codes are keys and the counts are values. Both are put into a list for later. 

In [46]:
count=Counter(fips)
keys=[]
values=[]

for i in count:
    keys.append(i)
    values.append(count[i])

##### Step 4) Leading Zeros

Both excel and python delete leading zeros. This is unhelpful whent you want to work with the full FIPS codes, so we will attach the zeros manually. This will also turn them all into strings. 

In [47]:
key=[]

for i in keys:
    if len(i)==4:
        i='0'+i
    key.append(i)

##### Step 5) DataFrame

Create a `pandas` dataframe from the FIPS codes and counts. Drop the last row because it was counting code-less CHCs.

In [48]:
df = pd.DataFrame()
df['FIPS Code'] = key
df['# of Health Centers'] = values
df = df.iloc[:2107]
LocationTable = df

## Merging and Final Dataset

Now that we have collected all of the data and put them into `pandas` dataframes, we can use the `merge` method to bring everything together. 

This data set will have an absurdly high number of rows. This is a consequence of the `merge` method duplicating rows but will not be a problem when we do the final merge.

In [52]:
# Merging all the socioeconomic data
PP = pd.merge(PopulationTable,PovertyRateTable, how = 'outer', on = 'FIPS Code')
PPO = pd.merge(PP,OldTable, how = 'outer', on = 'FIPS Code')
PPOI = pd.merge(PPO,IncomeRateTable, how = 'outer', on = 'FIPS Code')
PPOIH = pd.merge(PPOI,HomeValueTable, how = 'outer', on = 'FIPS Code')
PPOIHF = pd.merge(PPOIH,FoodStampTable, how = 'outer', on = 'FIPS Code')
PPOIHFC = pd.merge(PPOIHF,CollegeTable, how = 'outer', on = 'FIPS Code')
PPOIHFCH = pd.merge(PPOIHFC,HighschoolTable, how = 'outer', on = 'FIPS Code')
PPOIHFCH

,FIPS Code,Population,Poverty %,% Older than 65,Median Household Income,Median Home Value,Food Stamp Recipient %,Edu. < College %,Edu. < Highschool %
0,48301,74,13.3,24.3,64200.0,NaN,0.9,95.3,20.3
1,15005,86,NaN,14.0,NaN,NaN,NaN,77.2,12.7
2,48269,289,10.7,7.3,59100.0,38800.0,4.6,80.2,19.8
3,31005,421,11.9,21.9,44500.0,101500.0,0.4,70.1,4.9
4,31117,432,12.2,18.1,49300.0,137500.0,4.0,74.8,9.9
5,30069,453,15.1,22.1,43200.0,112000.0,4.6,74.7,1.9
6,31009,502,15.5,24.1,49000.0,85900.0,7.0,75.8,1.1
7,31115,537,16.0,25.9,47900.0,159100.0,4.3,77.1,1.7
8,35021,546,14.3,31.1,37000.0,67800.0,7.0,76.0,10.3
9,48261,564,16.2,14.7,38700.0,21300.0,17.7,93.5,58.7


In [53]:
#Merging Cardiovascular data and Socioeconomic data
CardioSocio = pd.merge(DiseaseRateTable,PPOIHFCH, how = 'outer', on = 'FIPS Code')
CardioSocio =  CardioSocio.drop_duplicates(subset = 'FIPS Code',keep = 'first')
CardioSocio
Total = pd.merge(CardioSocio,LocationTable,  on = 'FIPS Code', how = 'left')
Total['People/Health Centers'] = Total.apply(lambda row: float(row['Population'])/float(row['# of Health Centers']), axis=1)
Total

,FIPS Code,County Name,State Abbreviation,State Name,CV Death per 100K,Population,Poverty %,% Older than 65,Median Household Income,Median Home Value,Food Stamp Recipient %,Edu. < College %,Edu. < Highschool %,# of Health Centers,People/Health Centers
0,02013,Aleutians East,AK,Alaska,106.1,3338,14.8,8.2,64000.0,124700.0,3.8,88.5,14.2,6.0,556.333333
1,08097,Pitkin,CO,Colorado,133.8,17747,6.9,16.6,74600.0,593600.0,1.2,38.8,4.7,1.0,17747.000000
2,08037,Eagle,CO,Colorado,157.5,53726,7.9,9.4,79600.0,471100.0,2.7,54.9,10.1,5.0,10745.200000
3,08117,Summit,CO,Colorado,171.8,29722,7.9,11.3,70300.0,547700.0,3.0,52.2,6.6,8.0,3715.250000
4,02016,Aleutians West,AK,Alaska,184.7,5784,8.0,5.5,78200.0,238800.0,2.4,83.8,11.7,6.0,964.000000
5,08113,San Miguel,CO,Colorado,186.0,7804,10.1,11.3,67300.0,485000.0,4.3,45.9,6.4,1.0,7804.000000
6,08091,Ouray,CO,Colorado,210.9,4653,8.8,26.4,66800.0,420600.0,4.8,50.6,2.0,NaN,NaN
7,56039,Teton,WY,Wyoming,215.2,22923,7.3,12.7,84100.0,739100.0,0.6,45.9,4.9,NaN,NaN
8,51685,Manassas Park,VA,Virginia,221.1,16117,7.8,7.4,72400.0,263300.0,7.6,68.6,17.7,NaN,NaN
9,16013,Blaine,ID,Idaho,234.1,21583,9.8,16.5,66200.0,390300.0,3.5,59.6,11.3,NaN,NaN


We have now created the final dataset containing *FIPS Code*, *County Name*, *State Abbreviation*, *State Name*, *CV Death per 100K*, *Population*, *Poverty %*, *% Older than 65*, *Median Household Income*, *Median Home Value*, *Food Stamp Recipient %*,	*Edu. < College %*, *Edu. < Highschool %*, *# of Health Centers*, and *People/Health Centers*. There will be many NaNs considering the diversity of the data and their [sources](https://www.cdc.gov/dhdsp/maps/atlas/data-sources.html).

##### Writing Out

The *Total* dataframe is written out to a csv file. The FIPS codes will lose their leading zeros, but this should not affect it as a unique identifier.


In [74]:
Total.to_csv('dataset1.csv',encoding='utf-8',index=False)
